<a href="https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/Matrix_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scikit-surprise

     |████████████████████████████████| 11.8MB 7.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617538 sha256=a18749cdcf401e93f2e35cfba892c10e7dc1451840eaea0dd76f39c945dd9d8d
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [3]:
import os
os.mkdir('/content/matrix')
print('Folder created!')
os.chdir('/content/matrix')
print('Files are in this folder!')

Folder created!
Files are in this folder!


In [5]:
!python "MovieLens.py"
print('1 of 5: Done')
!python "RecommenderMetrics.py"
print('2 of 5: Done')
!python "EvaluationData.py"
print('3 of 5: Done')
!python "EvaluatedAlgorithm.py"
print('4 of 5: Done')
!python "Evaluator.py"
print('5 of 5: All Scripts Loaded!')

1 of 5: Done
2 of 5: Done
3 of 5: Done
4 of 5: Done
5 of 5: All Scripts Loaded!


In [4]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 14 22:35:07 2021

@author: Maribel
"""

from MovieLens import MovieLens
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator
from surprise.model_selection import GridSearchCV
from surprise.model_selection import RandomizedSearchCV

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

print("Searching for best parameters...")
#param_grid = {'n_epochs': [20, 30], 'lr_all': [0.005, 0.010],
#              'n_factors': [50, 100]}
#gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

#gs.fit(evaluationData)

param_grid = {'n_epochs': range(10, 31), 
              'lr_all': np.linspace(0.001, 0.10, 10),
              'n_factors': range(10, 100, 20),
              'reg_all': np.linspace(0.01, 0.1, 10)}

rs = RandomizedSearchCV(SVD, param_distributions=param_grid, measures=['rmse', 'mae'], 
                        cv=5, random_state=29, n_jobs = -1, n_iter=50)

rs.fit(evaluationData)

# best RMSE score
#print("Best RMSE score attained: ", gs.best_score['rmse'])
print("Best RMSE score attained: ", rs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
#print(gs.best_params['rmse'])
print(rs.best_params['rmse'])

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

#params = gs.best_params['rmse']
#SVDtuned = SVD(n_epochs = params['n_epochs'], lr_all = params['lr_all'], n_factors = params['n_factors'])

params = rs.best_params['rmse']
SVDtuned = SVD(n_epochs = params['n_epochs'], lr_all = params['lr_all'], 
               n_factors = params['n_factors'], reg_all = params['reg_all'])
evaluator.AddAlgorithm(SVDtuned, "SVD - Tuned")

SVDUntuned = SVD()
evaluator.AddAlgorithm(SVDUntuned, "SVD - Untuned")

# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)


Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Searching for best parameters...
Best RMSE score attained:  0.8553894737323182
{'n_epochs': 11, 'lr_all': 0.034, 'n_factors': 90, 'reg_all': 0.06000000000000001}
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD - Tuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD - Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.

In [6]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# SVD++ Untuned
SVDPlusPlus = SVDpp()
evaluator.AddAlgorithm(SVDPlusPlus, "SVD++Untuned")

# SVD++
SVDPlusPlusTuned = SVDpp(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)
#{'n_epochs': 11, 'lr_all': 0.034, 'n_factors': 90, 'reg_all': 0.06000000000000001}
evaluator.AddAlgorithm(SVDPlusPlusTuned, "SVD++Tuned")

# SVD++ Untuned
SVDPlusPlus = SVDpp()
evaluator.AddAlgorithm(SVDPlusPlus, "SVD++Untuned")

# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)


Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD++Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD++Tuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD++Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with fu

In [7]:
os.chdir('/content/sample_data')
print('Files are in this folder!')

Files are in this folder!


In [8]:
!python "MovieLens.py"
print('1 of 7: Done')
!python "RecommenderMetrics.py"
print('2 of 7: Done')
!python "EvaluationData.py"
print('3 of 7: Done')
!python "EvaluatedAlgorithm.py"
print('4 of 7: Done')
!python "Evaluator.py"
print('5 of 7: Done')
!python "RBMAlgorithm.py"
print('6 of 7: Done')
!python "RBM.py"
print('7 of 7: All Scripts Loaded!')

1 of 7: Done
2 of 7: Done
3 of 7: Done
4 of 7: Done
5 of 7: Done
2021-03-16 11:49:04.550402: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
6 of 7: Done
2021-03-16 11:49:14.348460: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
7 of 7: All Scripts Loaded!


In [9]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 15 23:32:16 2021

@author: Maribel
"""

# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from surprise import NormalPredictor
from Evaluator import Evaluator
from surprise.model_selection import GridSearchCV
from surprise.model_selection import RandomizedSearchCV

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

print("Searching for best parameters...")
param_grid = {'hiddenDim': range(10, 101, 5), 
              'learningRate': [0.001, 0.002, 0.005, np.linspace(0.01, 0.20, 20)]}
rs = RandomizedSearchCV(RBMAlgorithm, param_distributions = param_grid, n_jobs=-1,
                        measures=['rmse', 'mae'], cv=5, n_iter=50)

rs.fit(evaluationData)

# best RMSE score
print("Best RMSE score attained: ", rs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(rs.best_params['rmse'])

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

params = rs.best_params['rmse']
RBMtuned = RBMAlgorithm(hiddenDim = params['hiddenDim'], learningRate = params['learningRate'])
evaluator.AddAlgorithm(RBMtuned, "RBM - Tuned")

RBMUntuned = RBMAlgorithm()
evaluator.AddAlgorithm(RBMUntuned, "RBM - Untuned")

# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)


Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Searching for best parameters...


exception calling callback for <Future at 0x7ff09f522f50 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py", line 404, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
ModuleNotFoundError: No module named 'RBMAlgorithm'
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 792, in dispatch_next
    

BrokenProcessPool: ignored

***

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 14 22:35:07 2021

@author: Maribel
"""

from MovieLens import MovieLens
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator
from surprise.model_selection import GridSearchCV
from surprise.model_selection import RandomizedSearchCV

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

print("Searching for best parameters...")
#param_grid = {'n_epochs': [20, 30], 'lr_all': [0.005, 0.010],
#              'n_factors': [50, 100]}
#gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

#gs.fit(evaluationData)

param_grid = {'n_epochs': range(10, 31), 
              'lr_all': np.linspace(0.001, 0.20, 20),
              'n_factors': range(10, 100, 20),
              'reg_all': np.linspace(0.01, 0.1, 10)}

rsplus = RandomizedSearchCV(SVDpp, param_distributions=param_grid, measures=['rmse', 'mae'], 
                        cv=3, random_state=29, n_jobs = -1, n_iter=10)

rsplus.fit(evaluationData)

# best RMSE score
#print("Best RMSE score attained: ", gs.best_score['rmse'])
print("Best RMSE score attained: ", rsplus.best_score['rmse'])

# combination of parameters that gave the best RMSE score
#print(gs.best_params['rmse'])
print(rsplus.best_params['rmse'])

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

#params = gs.best_params['rmse']
#SVDtuned = SVD(n_epochs = params['n_epochs'], lr_all = params['lr_all'], n_factors = params['n_factors'])

params = rsplus.best_params['rmse']
SVDPlusTuned = SVDpp(n_epochs = params['n_epochs'], lr_all = params['lr_all'], 
               n_factors = params['n_factors'], reg_all = params['reg_all'])
evaluator.AddAlgorithm(SVDPlusTuned, "SVD++ - Tuned")

SVDPlusUntuned = SVDpp()
evaluator.AddAlgorithm(SVDPlusUntuned, "SVD++ - Untuned")

# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

# Fight!
evaluator.Evaluate(False)

evaluator.SampleTopNRecs(ml)


Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Searching for best parameters...


KeyboardInterrupt: ignored